In [18]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests as fdr
import warnings
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import het_breuschpagan

warnings.filterwarnings('ignore')


In [2]:
# Read and clean up ABCD data
# Set base directories
ABCD_base_dir = '/u/home/n/npeterse/BPM/abcd-data-release-5.1/'
BPM_base_dir = '/u/home/n/npeterse/BPM/'

# Load subject list and site data
ABCD_subj = pd.read_csv(os.path.join(BPM_base_dir, 'brainxpuberty/', 'pt_ids_abcd.csv'), header=None, names=['s'])
ABCD_site = pd.read_csv(os.path.join(ABCD_base_dir, 'core/abcd-general/abcd_y_lt.csv'), header=0)
ABCD_qc = pd.read_csv(os.path.join(ABCD_base_dir, 'core/imaging/mri_y_qc_incl.csv'), header=0)

# Load brain and puberty data
ABCD_ct = pd.read_csv(os.path.join(ABCD_base_dir, 'core/imaging/mri_y_smr_thk_dsk.csv'), header=0)
ABCD_sa = pd.read_csv(os.path.join(ABCD_base_dir, 'core/imaging/mri_y_smr_area_dsk.csv'), header=0)
ABCD_vol = pd.read_csv(os.path.join(ABCD_base_dir, 'core/imaging/mri_y_smr_vol_dsk.csv'), header=0)
ABCD_pds = pd.read_csv(os.path.join(ABCD_base_dir, 'core/physical-health/ph_y_pds.csv'), header=0)

# Keep relevant data for qc, pds, site
ABCD_qc = ABCD_qc[['src_subject_id', 'eventname', 'imgincl_t1w_include']]
ABCD_pds = ABCD_pds[['src_subject_id', 'eventname', 'menstrualcycle4_y']]
ABCD_site = ABCD_site[['src_subject_id', 'eventname', 'site_id_l']]

# Remove participants without pds data
ABCD_pds = ABCD_pds.dropna(subset=['menstrualcycle4_y'], how='all')
ABCD_pds = ABCD_pds[ABCD_pds.menstrualcycle4_y<2]

# Remove participants with poor quality img data
ABCD_qc = ABCD_qc[ABCD_qc.imgincl_t1w_include == 1]

# Sort data to ensure they're all in the same order
ABCD_qc = ABCD_qc.sort_values(by='src_subject_id', ascending=True)
ABCD_ct = ABCD_ct.sort_values(by='src_subject_id', ascending=True)
ABCD_sa = ABCD_sa.sort_values(by='src_subject_id', ascending=True)
ABCD_vol = ABCD_vol.sort_values(by='src_subject_id', ascending=True)
ABCD_pds = ABCD_pds.sort_values(by='src_subject_id', ascending=True)
ABCD_site = ABCD_site.sort_values(by='src_subject_id', ascending=True)

# Get data from 4-year follow-up arm
ABCD_qc_4y = ABCD_qc[ABCD_qc.eventname == '4_year_follow_up_y_arm_1']
ABCD_ct_4y = ABCD_ct[ABCD_ct.eventname == '4_year_follow_up_y_arm_1']
ABCD_sa_4y = ABCD_sa[ABCD_sa.eventname == '4_year_follow_up_y_arm_1']
ABCD_vol_4y = ABCD_vol[ABCD_vol.eventname == '4_year_follow_up_y_arm_1']
ABCD_pds_4y = ABCD_pds[ABCD_pds.eventname == '4_year_follow_up_y_arm_1']
ABCD_site_4y = ABCD_site[ABCD_site.eventname == '4_year_follow_up_y_arm_1']


In [3]:
# Figure out overlapping subject data across all measures
series1 = pd.Series(ABCD_ct_4y.src_subject_id)
series2 = pd.Series(ABCD_sa_4y.src_subject_id)
series3 = pd.Series(ABCD_vol_4y.src_subject_id)
series4 = pd.Series(ABCD_pds_4y.src_subject_id)
series5 = pd.Series(ABCD_qc_4y.src_subject_id)
subj_4y = set(series1) & set(series2) & set(series3) & set(series4) & set(series5)
subj_4y = pd.DataFrame(subj_4y)
subj_4y.columns = ['s']


In [4]:
# Get data just from participants of interest (i.e., participants with all data)
ABCD_ct_4y = ABCD_ct_4y[ABCD_ct_4y.src_subject_id.isin(subj_4y.s)]
ABCD_sa_4y = ABCD_sa_4y[ABCD_sa_4y.src_subject_id.isin(subj_4y.s)]
ABCD_vol_4y = ABCD_vol_4y[ABCD_vol_4y.src_subject_id.isin(subj_4y.s)]
ABCD_pds_4y = ABCD_pds_4y[ABCD_pds_4y.src_subject_id.isin(subj_4y.s)]
ABCD_site_4y = ABCD_site_4y[ABCD_site_4y.src_subject_id.isin(subj_4y.s)]


In [5]:
# Remove subject id and eventname columns
ABCD_ct_4y = ABCD_ct_4y.drop(columns=['src_subject_id', 'eventname', 'smri_thick_cdk_meanlh', 'smri_thick_cdk_meanrh', 'smri_thick_cdk_mean'])
ABCD_sa_4y = ABCD_sa_4y.drop(columns=['src_subject_id', 'eventname', 'smri_area_cdk_totallh', 'smri_area_cdk_totalrh', 'smri_area_cdk_total'])
ABCD_vol_4y = ABCD_vol_4y.drop(columns=['src_subject_id', 'eventname', 'smri_vol_cdk_totallh', 'smri_vol_cdk_totalrh', 'smri_vol_cdk_total'])
ABCD_pds_4y = ABCD_pds_4y.drop(columns=['src_subject_id', 'eventname'])
ABCD_site_4y = ABCD_site_4y.drop(columns=['src_subject_id', 'eventname'])


In [6]:
# Clean up dataframes to remove index
ABCD_ct_4y.reset_index(inplace=True)
ABCD_ct_4y = ABCD_ct_4y.drop(columns=['index'])

ABCD_sa_4y.reset_index(inplace=True)
ABCD_sa_4y = ABCD_sa_4y.drop(columns=['index'])

ABCD_vol_4y.reset_index(inplace=True)
ABCD_vol_4y = ABCD_vol_4y.drop(columns=['index'])

ABCD_pds_4y.reset_index(inplace=True)
ABCD_pds_4y = ABCD_pds_4y.drop(columns=['index'])

ABCD_site_4y.reset_index(inplace=True)
ABCD_site_4y = ABCD_site_4y.drop(columns=['index'])


In [13]:
rois_ct = ABCD_ct_4y.columns

ct_pvals = np.zeros(rois_ct.shape[0])

for i in range(rois_ct.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, 
                      ABCD_ct_4y[rois_ct[i]]])

    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'ct']
    ABCD_analyses['ct'] = pd.to_numeric(ABCD_analyses['ct'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    # Convert 'site' to a categorical variable
    ABCD_analyses['site'] = ABCD_analyses['site'].astype('category')

    # Include 'site' as a covariate in the GLM formula
    glm_model = smf.glm("ct ~ ocp + C(site)", data=ABCD_analyses, family=sm.families.Gaussian())
    glm_result = glm_model.fit()
    ct_pvals[i] = glm_result.pvalues['ocp']

# Print the uncorrected p-values from smallest to largest
print("Uncorrected p-values (cortical thickness):")
for roi, pval in sorted(zip(rois_ct, ct_pvals), key=lambda x: x[1]):
    print(f"{roi}: {pval}")

print(glm_result.summary())


Uncorrected p-values (cortical thickness):
smri_thick_cdk_paracnrh: 0.0001704232339434959
smri_thick_cdk_paracnlh: 0.0004230208939336686
smri_thick_cdk_ptcatelh: 0.006611005572512752
smri_thick_cdk_suplrh: 0.010621766790864691
smri_thick_cdk_precnlh: 0.015424363062994415
smri_thick_cdk_pcrh: 0.01668386119859675
smri_thick_cdk_supllh: 0.018448606019925658
smri_thick_cdk_locclh: 0.02117003675761538
smri_thick_cdk_sufrlh: 0.022413930550413862
smri_thick_cdk_loccrh: 0.02332571476175886
smri_thick_cdk_iftmlh: 0.032688738072645836
smri_thick_cdk_mobfrrh: 0.03447652320169142
smri_thick_cdk_mobfrlh: 0.040075728117880016
smri_thick_cdk_sufrrh: 0.04260893970684325
smri_thick_cdk_parstgrisrh: 0.04842824819106848
smri_thick_cdk_precnrh: 0.04885688327339865
smri_thick_cdk_postcnrh: 0.06085007826560542
smri_thick_cdk_cuneuslh: 0.06580321881213574
smri_thick_cdk_cdmdfrlh: 0.06922331862727013
smri_thick_cdk_lobfrlh: 0.06961792954960129
smri_thick_cdk_fusiformrh: 0.07334750063306589
smri_thick_cdk_iftm

In [14]:
rois_sa = ABCD_sa_4y.columns

sa_pvals = np.zeros(rois_sa.shape[0])

for i in range(rois_sa.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, 
                      ABCD_sa_4y[rois_sa[i]]])

    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'sa']
    ABCD_analyses['sa'] = pd.to_numeric(ABCD_analyses['sa'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    # Convert 'site' to a categorical variable
    ABCD_analyses['site'] = ABCD_analyses['site'].astype('category')

    # Include 'site' as a covariate in the GLM formula
    glm_model = smf.glm("sa ~ ocp + C(site)", data=ABCD_analyses, family=sm.families.Gaussian())
    glm_result = glm_model.fit()
    sa_pvals[i] = glm_result.pvalues['ocp']

# Print the uncorrected p-values labeled with corresponding ROIs
print("Uncorrected p-values (surface area):")
for roi, pval in sorted(zip(rois_sa, sa_pvals), key=lambda x: x[1]):
     print(f"{roi}: {pval}")

print(glm_result.summary())


Uncorrected p-values (surface area):
smri_area_cdk_postcnlh: 0.024675255430653267
smri_area_cdk_trvtmlh: 0.04944153424022199
smri_area_cdk_precnlh: 0.07170521290855082
smri_area_cdk_rracatelh: 0.08471318507738207
smri_area_cdk_precnrh: 0.14252550126523772
smri_area_cdk_cdmdfrlh: 0.14672517429217535
smri_area_cdk_sutmlh: 0.15465896566619494
smri_area_cdk_ehinalrh: 0.1659741395042883
smri_area_cdk_postcnrh: 0.18562014676134897
smri_area_cdk_parahpallh: 0.20577851694391758
smri_area_cdk_mdtmlh: 0.22281840843921652
smri_area_cdk_mobfrrh: 0.24153415422725089
smri_area_cdk_tmpolerh: 0.25172558369681153
smri_area_cdk_ifpllh: 0.2671070376762218
smri_area_cdk_pcrh: 0.2905472302791592
smri_area_cdk_insularh: 0.30258430339754205
smri_area_cdk_ehinallh: 0.3187319042415625
smri_area_cdk_paracnlh: 0.3221858873807445
smri_area_cdk_linguallh: 0.3238182198195516
smri_area_cdk_sufrrh: 0.34892389413740543
smri_area_cdk_frpolelh: 0.362017201054859
smri_area_cdk_ihcatelh: 0.3687358317470527
smri_area_cdk_c

In [15]:
rois_vol = ABCD_vol_4y.columns

vol_pvals = np.zeros(rois_vol.shape[0])

for i in range(rois_vol.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, 
                      ABCD_vol_4y[rois_vol[i]]])

    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'vol']
    ABCD_analyses['vol'] = pd.to_numeric(ABCD_analyses['vol'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    # Convert 'site' to a categorical variable
    ABCD_analyses['site'] = ABCD_analyses['site'].astype('category')

    # Include 'site' as a covariate in the GLM formula
    glm_model = smf.glm("vol ~ ocp + C(site)", data=ABCD_analyses, family=sm.families.Gaussian())
    glm_result = glm_model.fit()
    vol_pvals[i] = glm_result.pvalues['ocp']

# Print the uncorrected p-values labeled with corresponding ROIs
print("Uncorrected p-values (volume):")
for roi, pval in sorted(zip(rois_vol, vol_pvals), key=lambda x: x[1]):
     print(f"{roi}: {pval}")

print(glm_result.summary())


Uncorrected p-values (volume):
smri_vol_cdk_precnlh: 0.002515423865296058
smri_vol_cdk_paracnlh: 0.004075825886302616
smri_vol_cdk_precnrh: 0.005735122648750854
smri_vol_cdk_postcnlh: 0.009886211888629778
smri_vol_cdk_paracnrh: 0.027299419820972472
smri_vol_cdk_postcnrh: 0.031100064806403406
smri_vol_cdk_mobfrrh: 0.03542299001630976
smri_vol_cdk_cdmdfrlh: 0.04156843764514395
smri_vol_cdk_sufrrh: 0.04777984563763633
smri_vol_cdk_trvtmlh: 0.05981792513949443
smri_vol_cdk_sufrlh: 0.06821202556187841
smri_vol_cdk_ptcatelh: 0.08682593882813432
smri_vol_cdk_tmpolerh: 0.09740338002308269
smri_vol_cdk_parsobisrh: 0.10468616629039683
smri_vol_cdk_ptcaterh: 0.11067907120490085
smri_vol_cdk_frpolerh: 0.1155743529889388
smri_vol_cdk_rracatelh: 0.11764128973141008
smri_vol_cdk_linguallh: 0.11810892341997113
smri_vol_cdk_ifplrh: 0.14397962182995458
smri_vol_cdk_sutmlh: 0.14859064014356227
smri_vol_cdk_iftmlh: 0.15278223736549076
smri_vol_cdk_mdtmrh: 0.18531674036760037
smri_vol_cdk_ehinalrh: 0.19973

In [16]:
# Function to calculate Cohen's d
def cohen_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    pooled_std = np.sqrt(((n1 - 1) * std1 ** 2 + (n2 - 1) * std2 ** 2) / (n1 + n2 - 2))
    d = (mean1 - mean2) / pooled_std
    return d

# Cohen's d calculation for cortical thickness, surface area, and volume
cohen_d_results = {
    'ROI': [],
    'Cohen_d_ct': [],
    'Cohen_d_sa': [],
    'Cohen_d_vol': []
}

# Calculate Cohen's d for cortical thickness
for roi in ABCD_ct_4y.columns:
    group1 = ABCD_ct_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 0]
    group2 = ABCD_ct_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 1]
    cohen_d_results['ROI'].append(roi)
    cohen_d_results['Cohen_d_ct'].append(cohen_d(group1, group2))

# Calculate Cohen's d for surface area
for roi in ABCD_sa_4y.columns:
    group1 = ABCD_sa_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 0]
    group2 = ABCD_sa_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 1]
    cohen_d_results['Cohen_d_sa'].append(cohen_d(group1, group2))

# Calculate Cohen's d for volume
for roi in ABCD_vol_4y.columns:
    group1 = ABCD_vol_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 0]
    group2 = ABCD_vol_4y[roi][ABCD_pds_4y['menstrualcycle4_y'] == 1]
    cohen_d_results['Cohen_d_vol'].append(cohen_d(group1, group2))

# Convert results to a DataFrame
cohen_d_df = pd.DataFrame(cohen_d_results)

# Display the results
print("Cohen's d for Cortical Thickness, Surface Area, and Volume:")
display(cohen_d_df)

# Save the results to a CSV file
cohen_d_df.to_csv('cohen_d_results.csv', index=False)
print("Cohen's d results have been saved to 'cohen_d_results.csv'.")


Cohen's d for Cortical Thickness, Surface Area, and Volume:


,ROI,Cohen_d_ct,Cohen_d_sa,Cohen_d_vol
0,smri_thick_cdk_banksstslh,-0.167692,-0.128360,-0.098349
1,smri_thick_cdk_cdacatelh,0.121086,-0.100975,-0.065742
2,smri_thick_cdk_cdmdfrlh,0.230540,0.164696,0.239469
3,smri_thick_cdk_cuneuslh,0.176350,-0.018125,0.057816
4,smri_thick_cdk_ehinallh,0.056141,-0.176078,-0.120835
...,...,...,...,...
63,smri_thick_cdk_smrh,0.105328,-0.118554,-0.065368
64,smri_thick_cdk_frpolerh,0.116397,0.018678,0.119495
65,smri_thick_cdk_tmpolerh,-0.034056,-0.192629,-0.245639
66,smri_thick_cdk_trvtmrh,0.134458,0.008100,0.050696


Cohen's d results have been saved to 'cohen_d_results.csv'.


In [19]:
# Function to run GLM and check assumptions
def run_glm_and_check_assumptions(data, response_variable, roi_name):
    # Define the GLM model
    formula = f"{response_variable} ~ ocp"
    glm_model = smf.glm(formula=formula, data=data, family=sm.families.Binomial())
    
    # Fit the model
    glm_result = glm_model.fit()
    
    # Print the summary of the GLM model
   # print(f"Results for {roi_name}")
   # print(glm_result.summary())
    
    # Shapiro-Wilk test for normality of residuals
    residuals = glm_result.resid_response
    shapiro_test = stats.shapiro(residuals)
    # print(f'Shapiro-Wilk test p-value for {roi_name}: {shapiro_test.pvalue}')
    
    # Breusch-Pagan test for homoscedasticity
    exog = glm_result.model.exog
    bp_test = het_breuschpagan(residuals, exog)
    # print(f'Breusch-Pagan test statistic for {roi_name}: {bp_test[0]}')
    # print(f'Breusch-Pagan test p-value for {roi_name}: {bp_test[1]}')
    
    # Print ROI name and specific failed assumption
    if shapiro_test.pvalue < 0.05:
        print(f'{roi_name} failed the Shapiro-Wilk normality test.')
    if bp_test[1] < 0.05:
        print(f'{roi_name} failed the Breusch-Pagan homoscedasticity test.')

# Example usage
for i in range(len(rois_ct)):
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_ct_4y[rois_ct[i]]])
    ABCD_analyses = pd.DataFrame(temp.T, columns=['subj', 'ocp', 'site', 'ct'])
    ABCD_analyses['ct'] = pd.to_numeric(ABCD_analyses['ct'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    # Run GLM and check assumptions
    run_glm_and_check_assumptions(ABCD_analyses, 'ct', rois_ct[i])


smri_thick_cdk_cdacatelh failed the Shapiro-Wilk normality test.
smri_thick_cdk_cdmdfrlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_cuneuslh failed the Shapiro-Wilk normality test.
smri_thick_cdk_ehinallh failed the Shapiro-Wilk normality test.
smri_thick_cdk_ifpllh failed the Shapiro-Wilk normality test.
smri_thick_cdk_iftmlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_ihcatelh failed the Shapiro-Wilk normality test.
smri_thick_cdk_lobfrlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_mobfrlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_mobfrlh failed the Breusch-Pagan homoscedasticity test.
smri_thick_cdk_mdtmlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_parahpallh failed the Shapiro-Wilk normality test.
smri_thick_cdk_paracnlh failed the Breusch-Pagan homoscedasticity test.
smri_thick_cdk_parsobislh failed the Shapiro-Wilk normality test.
smri_thick_cdk_postcnlh failed the Shapiro-Wilk normality test.
smri_thick_cdk_ptcatelh fail

In [20]:
# since there are lots of failed assumptions, we will attempt a robust regression instead

# Import the necessary library for Robust Linear Model
from statsmodels.robust.robust_linear_model import RLM

# Run robust regression and collect p-values
def run_robust_regression(data, roi_column):
    formula = f"{roi_column} ~ ocp"
    model = smf.rlm(formula, data=data, M=sm.robust.norms.HuberT())
    result = model.fit()
    return result

# List of ROIs and corresponding p-values
rois_ct = ABCD_ct_4y.columns
rois_sa = ABCD_sa_4y.columns
rois_vol = ABCD_vol_4y.columns

ct_robust_pvals = np.zeros(rois_ct.shape[0])
sa_robust_pvals = np.zeros(rois_sa.shape[0])
vol_robust_pvals = np.zeros(rois_vol.shape[0])

# Cortical thickness robust regression
for i in range(rois_ct.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_ct_4y[rois_ct[i]]])
    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'ct']
    ABCD_analyses['ct'] = pd.to_numeric(ABCD_analyses['ct'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    result = run_robust_regression(ABCD_analyses, 'ct')
    ct_robust_pvals[i] = result.pvalues['ocp']

# Surface area robust regression
for i in range(rois_sa.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_sa_4y[rois_sa[i]]])
    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'sa']
    ABCD_analyses['sa'] = pd.to_numeric(ABCD_analyses['sa'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    result = run_robust_regression(ABCD_analyses, 'sa')
    sa_robust_pvals[i] = result.pvalues['ocp']

# Volume robust regression
for i in range(rois_vol.shape[0]): 
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_vol_4y[rois_vol[i]]])
    ABCD_analyses = pd.DataFrame(temp.T)
    ABCD_analyses.columns = ['subj', 'ocp', 'site', 'vol']
    ABCD_analyses['vol'] = pd.to_numeric(ABCD_analyses['vol'])
    ABCD_analyses['ocp'] = pd.to_numeric(ABCD_analyses['ocp'])
    
    result = run_robust_regression(ABCD_analyses, 'vol')
    vol_robust_pvals[i] = result.pvalues['ocp']

# Corrected p-values
ct_robust_pvals_corr = fdr(ct_robust_pvals, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
sa_robust_pvals_corr = fdr(sa_robust_pvals, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
vol_robust_pvals_corr = fdr(vol_robust_pvals, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]

# Printing results
ct_robust_results = pd.DataFrame(ct_robust_pvals_corr[ct_robust_pvals_corr<0.05], rois_ct[ct_robust_pvals_corr<0.05].values)
ct_robust_results.columns = ['corrected p values']
print('Cortical Thickness Robust Regression Results')
print(ct_robust_results)

sa_robust_results = pd.DataFrame(sa_robust_pvals_corr[sa_robust_pvals_corr<0.05], rois_sa[sa_robust_pvals_corr<0.05].values)
sa_robust_results.columns = ['corrected p values']
print('Surface Area Robust Regression Results')
print(sa_robust_results)

vol_robust_results = pd.DataFrame(vol_robust_pvals_corr[vol_robust_pvals_corr<0.05], rois_vol[vol_robust_pvals_corr<0.05].values)
vol_robust_results.columns = ['corrected p values']
print('Volume Robust Regression Results')
print(vol_robust_results)




Cortical Thickness Robust Regression Results
                         corrected p values
smri_thick_cdk_paracnlh            0.011864
smri_thick_cdk_paracnrh            0.011864
Surface Area Robust Regression Results
Empty DataFrame
Columns: [corrected p values]
Index: []
Volume Robust Regression Results
Empty DataFrame
Columns: [corrected p values]
Index: []


In [21]:
# and just to be extra sure of our results, we'll also try both a robust regression, a glm, and a normal
# ordinary least squares after a log transformation to normalize the data

# Apply log transformation
ABCD_ct_4y_log = np.log1p(ABCD_ct_4y)  # Adding 1 to avoid log(0)
ABCD_sa_4y_log = np.log1p(ABCD_sa_4y)
ABCD_vol_4y_log = np.log1p(ABCD_vol_4y)

# run all regressions and collect p-values
def run_regressions(data, response_variable, predictor_variable):
    pvals = {}

    # Robust Regression
    robust_model = smf.rlm(f"{response_variable} ~ {predictor_variable}", data=data)
    robust_result = robust_model.fit()
    pvals['robust'] = robust_result.pvalues[predictor_variable]

    # OLS Regression
    ols_model = smf.ols(f"{response_variable} ~ {predictor_variable}", data=data)
    ols_result = ols_model.fit()
    pvals['ols'] = ols_result.pvalues[predictor_variable]

    # GLM Regression
    glm_model = smf.glm(f"{response_variable} ~ {predictor_variable}", data=data, family=sm.families.Gaussian())
    glm_result = glm_model.fit()
    pvals['glm'] = glm_result.pvalues[predictor_variable]

    return pvals

# Prepare to collect p-values
ct_pvals_robust = []
ct_pvals_ols = []
ct_pvals_glm = []

sa_pvals_robust = []
sa_pvals_ols = []
sa_pvals_glm = []

vol_pvals_robust = []
vol_pvals_ols = []
vol_pvals_glm = []

# Run regressions for cortical thickness
for roi in ABCD_ct_4y_log.columns:
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_ct_4y_log[roi]])
    ABCD_analyses_log = pd.DataFrame(temp.T)
    ABCD_analyses_log.columns = ['subj', 'ocp', 'site', 'ct_log']
    ABCD_analyses_log['ct_log'] = pd.to_numeric(ABCD_analyses_log['ct_log'])
    ABCD_analyses_log['ocp'] = pd.to_numeric(ABCD_analyses_log['ocp'])

    pvals = run_regressions(ABCD_analyses_log, 'ct_log', 'ocp')
    ct_pvals_robust.append(pvals['robust'])
    ct_pvals_ols.append(pvals['ols'])
    ct_pvals_glm.append(pvals['glm'])

# Run regressions for surface area
for roi in ABCD_sa_4y_log.columns:
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_sa_4y_log[roi]])
    ABCD_analyses_log = pd.DataFrame(temp.T)
    ABCD_analyses_log.columns = ['subj', 'ocp', 'site', 'sa_log']
    ABCD_analyses_log['sa_log'] = pd.to_numeric(ABCD_analyses_log['sa_log'])
    ABCD_analyses_log['ocp'] = pd.to_numeric(ABCD_analyses_log['ocp'])

    pvals = run_regressions(ABCD_analyses_log, 'sa_log', 'ocp')
    sa_pvals_robust.append(pvals['robust'])
    sa_pvals_ols.append(pvals['ols'])
    sa_pvals_glm.append(pvals['glm'])

# Run regressions for volume
for roi in ABCD_vol_4y_log.columns:
    temp = np.vstack([subj_4y.s, ABCD_pds_4y.menstrualcycle4_y, ABCD_site_4y.site_id_l, ABCD_vol_4y_log[roi]])
    ABCD_analyses_log = pd.DataFrame(temp.T)
    ABCD_analyses_log.columns = ['subj', 'ocp', 'site', 'vol_log']
    ABCD_analyses_log['vol_log'] = pd.to_numeric(ABCD_analyses_log['vol_log'])
    ABCD_analyses_log['ocp'] = pd.to_numeric(ABCD_analyses_log['ocp'])

    pvals = run_regressions(ABCD_analyses_log, 'vol_log', 'ocp')
    vol_pvals_robust.append(pvals['robust'])
    vol_pvals_ols.append(pvals['ols'])
    vol_pvals_glm.append(pvals['glm'])

# Apply FDR correction
ct_pvals_robust_corr = fdr(ct_pvals_robust, alpha=0.05, method='fdr_bh')[1]
ct_pvals_ols_corr = fdr(ct_pvals_ols, alpha=0.05, method='fdr_bh')[1]
ct_pvals_glm_corr = fdr(ct_pvals_glm, alpha=0.05, method='fdr_bh')[1]

sa_pvals_robust_corr = fdr(sa_pvals_robust, alpha=0.05, method='fdr_bh')[1]
sa_pvals_ols_corr = fdr(sa_pvals_ols, alpha=0.05, method='fdr_bh')[1]
sa_pvals_glm_corr = fdr(sa_pvals_glm, alpha=0.05, method='fdr_bh')[1]

vol_pvals_robust_corr = fdr(vol_pvals_robust, alpha=0.05, method='fdr_bh')[1]
vol_pvals_ols_corr = fdr(vol_pvals_ols, alpha=0.05, method='fdr_bh')[1]
vol_pvals_glm_corr = fdr(vol_pvals_glm, alpha=0.05, method='fdr_bh')[1]

# Collect the results
ct_results = pd.DataFrame({
    'ROI': ABCD_ct_4y_log.columns,
    'robust_pvalue': ct_pvals_robust,
    'ols_pvalue': ct_pvals_ols,
    'glm_pvalue': ct_pvals_glm,
    'robust_pvalue_corr': ct_pvals_robust_corr,
    'ols_pvalue_corr': ct_pvals_ols_corr,
    'glm_pvalue_corr': ct_pvals_glm_corr
})

sa_results = pd.DataFrame({
    'ROI': ABCD_sa_4y_log.columns,
    'robust_pvalue': sa_pvals_robust,
    'ols_pvalue': sa_pvals_ols,
    'glm_pvalue': sa_pvals_glm,
    'robust_pvalue_corr': sa_pvals_robust_corr,
    'ols_pvalue_corr': sa_pvals_ols_corr,
    'glm_pvalue_corr': sa_pvals_glm_corr
})

vol_results = pd.DataFrame({
    'ROI': ABCD_vol_4y_log.columns,
    'robust_pvalue': vol_pvals_robust,
    'ols_pvalue': vol_pvals_ols,
    'glm_pvalue': vol_pvals_glm,
    'robust_pvalue_corr': vol_pvals_robust_corr,
    'ols_pvalue_corr': vol_pvals_ols_corr,
    'glm_pvalue_corr': vol_pvals_glm_corr
})

# Display the results
print("Cortical Thickness Results (log-transformed, uncorrected and then corrected):")
print(ct_results[ct_results['robust_pvalue_corr'] < 0.05])

print("\nSurface Area Results (log-transformed, uncorrected and then corrected):")
print(sa_results[sa_results['robust_pvalue_corr'] < 0.05])

print("\nVolume Results (log-transformed, uncorrected and then corrected):")
print(vol_results[vol_results['robust_pvalue_corr'] < 0.05])


Cortical Thickness Results (log-transformed, uncorrected and then corrected):
                        ROI  robust_pvalue  ols_pvalue  glm_pvalue  \
15  smri_thick_cdk_paracnlh       0.000153    0.000281    0.000270   
49  smri_thick_cdk_paracnrh       0.000325    0.000146    0.000139   

    robust_pvalue_corr  ols_pvalue_corr  glm_pvalue_corr  
15            0.010419         0.009568         0.009186  
49            0.011064         0.009568         0.009186  

Surface Area Results (log-transformed, uncorrected and then corrected):
Empty DataFrame
Columns: [ROI, robust_pvalue, ols_pvalue, glm_pvalue, robust_pvalue_corr, ols_pvalue_corr, glm_pvalue_corr]
Index: []

Volume Results (log-transformed, uncorrected and then corrected):
Empty DataFrame
Columns: [ROI, robust_pvalue, ols_pvalue, glm_pvalue, robust_pvalue_corr, ols_pvalue_corr, glm_pvalue_corr]
Index: []


In [22]:
# Correct p-values using FDR
ct_pvals_corr = fdr(ct_pvals_glm, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
sa_pvals_corr = fdr(sa_pvals_glm, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
vol_pvals_corr = fdr(vol_pvals_glm, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]

# Display results
ct_results = pd.DataFrame(ct_pvals_corr[ct_pvals_corr<0.05], rois_ct[ct_pvals_corr<0.05].values)
ct_results.columns = ['corrected p values']
print('Cortical Thickness Results')
display(ct_results)

sa_results = pd.DataFrame(sa_pvals_corr[sa_pvals_corr<0.05], rois_sa[sa_pvals_corr<0.05].values)
sa_results.columns = ['corrected p values']
print('Surface Area Results')
display(sa_results)

vol_results = pd.DataFrame(vol_pvals_corr[vol_pvals_corr<0.05], rois_vol[vol_pvals_corr<0.05].values)
vol_results.columns = ['corrected p values']
print('Volume Results')
display(vol_results)


Cortical Thickness Results


,corrected p values
smri_thick_cdk_paracnlh,0.009186
smri_thick_cdk_paracnrh,0.009186


Surface Area Results


,corrected p values


Volume Results


,corrected p values
